---

# CSCI 3202, Fall 2022
# Homework 2: MDP & Reinforcement Learning
# Due: Friday September 9, 2022 at 6:00 PM

<br> 

### Your name: Giuliano Costa

<br> 

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict

# added packages
import heapq
from matplotlib import colors



---

Consider a **cube** state space defined by $0 \le x, y, z \le L$. Suppose you are piloting/programming a drone to learn how to land on a platform at the center of the $z=0$ surface (the bottom). Some assumptions:
* In this discrete world, if the drone is at $(x,y,z)$ it means that the box is centered at $(x,y,z)$. There are boxes (states) centered at $(x,y,z)$ for all $0 \le x,y,z \le L$. Each state is a 1 unit cube. 
* In this world, $L$ is always an even value.
* All of the states with $z=0$ are terminal states.
* The state at the center of the bottom of the cubic state space is the landing pad. For example, when $L=4$, the landing pad is at $(x,y,z) = (2,2,0)$.
* All terminal states ***except*** the landing pad have a reward of -1. The landing pad has a reward of +1.
* All non-terminal states have a living reward of -0.01.
* The drone takes up exactly 1 cubic unit, and begins in a random non-terminal state.
* The available actions in non-terminal states include moving exactly 1 unit Up (+z), Down (-z), North (+y), South (-y), East (+x) or West (-x). In a terminal state, the training episode should end.

#### Part A
How many states would be in the discrete state space if $L=2$? Explain your reasoning.

*Your answer here:*

State Space
$$ L = 2 + 1$$
$$ Z_{states}: 0, 1, 2 $$
$$ States_{num} = L^{Z} $$
$$ States_{num} = 3^3 $$
$$ \text{Total States} = 27 $$




#### Part B
Write a class `MDPLanding` to represent the Markov decision process for this drone. Include methods for:
1. `actions(state)`, which should return a list of all actions available from the given state
2. `reward(state)`, which should return the reward for the given state
3. `result(state, action)`, which should return the resulting state of doing the given action in the given state

and attributes for:
1. `states`, a list of all the states in the state space, where each state is represented as an $(x,y,z)$ tuple
2. `terminal_states`, a dictionary where keys are the terminal state tuples and the values are the rewards associated with those terminal states
3. `default_reward`, a scalar for the reward associated with non-terminal states
4. `all_actions`, a list of all possible actions (Up, Down, North, South, East, West)
5. `discount`, the discount factor (use $\gamma = 0.999$ for this entire problem)

How you feed arguments/information into the class constructor is up to you.

Note that actions are *deterministic* here.  The drone does not need to include transition probabilities for outcomes of particular actions. What the drone does need to learn, however, is where the landing pad is, and how to get there from any initial state.

Before moving on to Part C, we recommend that you test that your MDPLanding code is set up correctly. Write unit tests that display the actions for a given state, rewards, results, etc. This will help you identify errors in your implementation and save you a lot of debugging time later.

In [2]:
# Solution:
import itertools as it

class MDPLanding():
    def __init__(self, L):
        self.L = L
        self.states = list(it.product(range(L+1), repeat=3))
        self.terminal_states = {}
        
        for s in self.states:
            if s[2] == 0: #if z coordinate of tuple is 0, bottom of cube
                if(s[0] == L/2 ) and (s[1] == L/2): #LANDING PAD
                    self.terminal_states[s] = 1
                else:                               #OTHER TERMINAL STATES
                    self.terminal_states[s] = -1
        #self.terminal_states[(int(L/2), int(L/2) , 0)] = -1
        self.default_reward = -0.01
        self.all_actions = [(0,1,0), (0,-1,0), (-1,0,0), (1,0,0), (0,0,-1), (0,0,1)]
        self.discount = 0.999        
        
    #Return a set of actions, based on current (xyz) coord    
    def actions(self, state):
        #print(state)
        avail = []
        
        if(state[2] == 0): #if at the bottom
            return avail
        else:
            for move in self.all_actions:
                xyz = (state[0] + move[0], state[1] + move[1], state[2] + move[2])
                
                if xyz[0] < 0 or xyz[0] > self.L: #If change in x coord is less than 0, or greater than bounds, 
                    #print("x Not out of bounds \n")
                    continue
                elif xyz[1] < 0 or xyz[1] > self.L: #y
                    #print("y Not out of bounds \n")
                    continue
                elif xyz[2] < 0 or xyz[2] > self.L: #z
                    #print("z Not out of bounds \n")
                    continue
                else:
                    avail.append(move)
            return avail       
        # if(state in self.terminal_states): #If term state, return false
        #     return 0          
        # else: #Otherwise return all_actions
        #     return self.all_actions        
    
    def reward(self, state): #All terminal states have a reward of -1, except the landing pad
        #print("TERMINAL STATES: ", self.terminal_states ,"\n")
        if(state[2] == 0): #LANDING PAD
            if(state[0] == self.L/2 ) and (state[1] == self.L/2):
                #print("Reward 1\n")                
                return 1
            else:
                #print("Reward -1\n")
                return -1
        else:
            #print("Reward -0.01\n")
            return self.default_reward
        
    def result(self, state, action):
        if (state == None) or (action == None):
            return ()
        else:
            return (state[0] + action[0], state[1] + action[1], state[2] + action[2])
                                                           
        


In [3]:
test = MDPLanding(L=4)
len(test.states)
#test.actions((0,1,2))
#test.actions((0,-2,0))
#test.terminal_states
test.actions((4,4,4))
test.reward((4,4,4))
s = (4,4,4)
action = (0,-1,0)
test.reward( test.result(s, action) )

-0.01

#### Part C
Write a function to implement **policy iteration** for this drone landing MDP. Create an MDP environment to represent the $L=4$ case.

Use your function to find an optimal policy for your new MDP environment. Check (by printing to screen) that the policy for the following states are what you expect, and **comment on the results**:
1. $(2,2,1)$
1. $(0,2,1)$
1. $(2,0,1)$

The policy for each of these states is the action that the agent should take in that state. 

In [4]:
mdpenv = MDPLanding(L=4)

#Return utility (reward), if action were to be performed
def expected_utility(action, s, U, mdp):
    #print("expected_utility: ", mdp.reward(mdp.result(s, action)))
    #print("2. With this state", s , "\n")    
    #print("3. With this action", action , "\n")
    
    return mdp.reward( mdp.result(s, action))
 
#def policyEvaluation() Calculate the utility of each state U[s], if it were to be evaluated
def policy_evaluation(pi, U, mdp):
    #print(pi)
    for s in mdp.states:
        #if(s[2] != 0):
        #print("1 .Evalute this policy:", s, "With action:", pi[s])
        U[s] = mdp.default_reward + mdp.discount * expected_utility(pi[s], s, U, mdp) #* expectedUtlity(policy[state], state, utils, mdp))
    return U

def choose_best_action(actions, s, U, mdp):
    max_U = 0
    curr_U = 0
    action_to_choose = (0,0,0)
    #print("Choose best action \n")
    for a in actions:
        curr_U = expected_utility(a, s, U, mdp)
        if curr_U > max_U:
            max_U = curr_U
            action_to_choose = a
    return action_to_choose
        
#def expectedUtility() Expected utility of doing an action in a state (s), using mdp) 
def policy_iteration(mdp):
    U = {} #utilities
    pi = {} #policies
    states = [] #non terminal states
    
    for s in mdp.states:
        U[s] = 0  
        #if s[2] != 0:
    #A dictionary of utilities for states in mdp.state, {State: Utility}
    #Calculate Utility of each action if it were to be executed

    #Pi dictionary of {key: value}
    #Where key is a state (Up/Down etc...) and  value is an action    
    
    for s in mdp.states:
        states.append(s)        
        #if s[2] != 0:
    #print(len(mdp.states))
    #print(len(states))
    
    for s in mdp.states:
        if s[2] == 0:
            pi[s] = (0,0,0)
        else:
            pi[s] = mdp.actions(s)[np.random.choice(len(mdp.actions(s)), replace=True )]
    #print(pi)
    #print(len(U))
    while True:
        U = policy_evaluation(pi, U, mdp)
        unchanged = True
                
        for i in states: #for each state
            a = choose_best_action(mdp.actions(i), i, U, mdp) #), expected_utility(a, s, U, mdp))
            if a != pi[i]: #change {State: Action}
                #if(p[s] = -0.1)
                print("CHANGING POLICY: ", i, "TO ACTION:", pi[i]) #(0,-1,0)
                pi[i] = a
                unchanged = False
        if unchanged: 
            print("Unchanged")
            break
    return 


In [5]:
#mdpenv.actions((0,1,0))
policy_iteration(mdpenv)
#np.random.choice(len(mdpenv.actions((0,1,0))))

CHANGING POLICY:  (0, 0, 1) TO ACTION: (1, 0, 0)
CHANGING POLICY:  (0, 0, 2) TO ACTION: (1, 0, 0)
CHANGING POLICY:  (0, 0, 3) TO ACTION: (0, 0, -1)
CHANGING POLICY:  (0, 0, 4) TO ACTION: (0, 1, 0)
CHANGING POLICY:  (0, 1, 1) TO ACTION: (1, 0, 0)
CHANGING POLICY:  (0, 1, 2) TO ACTION: (0, -1, 0)
CHANGING POLICY:  (0, 1, 3) TO ACTION: (0, -1, 0)
CHANGING POLICY:  (0, 1, 4) TO ACTION: (0, 1, 0)
CHANGING POLICY:  (0, 2, 1) TO ACTION: (0, 0, -1)
CHANGING POLICY:  (0, 2, 2) TO ACTION: (0, 0, -1)
CHANGING POLICY:  (0, 2, 3) TO ACTION: (0, 0, -1)
CHANGING POLICY:  (0, 2, 4) TO ACTION: (1, 0, 0)
CHANGING POLICY:  (0, 3, 1) TO ACTION: (1, 0, 0)
CHANGING POLICY:  (0, 3, 2) TO ACTION: (0, -1, 0)
CHANGING POLICY:  (0, 3, 3) TO ACTION: (0, 1, 0)
CHANGING POLICY:  (0, 3, 4) TO ACTION: (0, -1, 0)
CHANGING POLICY:  (0, 4, 1) TO ACTION: (0, -1, 0)
CHANGING POLICY:  (0, 4, 2) TO ACTION: (0, 0, -1)
CHANGING POLICY:  (0, 4, 3) TO ACTION: (0, -1, 0)
CHANGING POLICY:  (0, 4, 4) TO ACTION: (1, 0, 0)
CHANGING 

For each of the states:
1. $(2,2,1)$ 
1. $(0,2,1)$
1. $(2,0,1)$

The resulting policies/actions are:
1. move down $(0,0,-1)$ which ends in $(2,2,0)$, the landing pad.
1. move down $(0,0,-1)$ which ends on a terminal state. Incorrect.
1. move down $(0,0,-1)$ which ends on a terminal state. Incorrect.

#### Part D
Provide an example of a non-deterministic transition that could be included in your code in Part C. Describe the function. How would you modify your code to handle a non-deterministic transition function?

*Your answer here*

Since there are a finite number of states in our MDP model, it can be considered deterministic. We know the probability of the state changing, since we are aware of the reward that action would result in. But when information is not known, the model changes to a partially observable MDP, POMDP. The code could be changed so that actions are not definitive, probabilities could be assigned to actions such that the probability to satisfy a selected action is less than 1. In our case it's always a 100% probability of success for an action.


#### Part E
Describe the main differences between **policy iteration** and **value iteration**? How would your code change in Part C to convert it to **value iteration**?

*Your answer here:*

Policy iteration contains two parts: policy evaluation and policy improvement. In a sort of feedback loop, policies are evaluated, then improved, evaluated again, and so on, until a policy converges (no changes are made). 

Value iteration on the other hand, begins with an arbitrary set of values for the utilities. It evaluates each policy only once and selects a max utility value. For each time step a value, $\delta$, is computed. This measures the change in utilities from the previous time step. If it is less than epsilon (a predefined constant) the algorithm terminates. This is simpler but more computationally expensive than policy iteration. 

To change this algorithm, I would move the bulk of my policy_evaluation function into value_iteration(). I would include the delta/epsilon calculation in the main for loop. As well as remove the expected_utility() function. It would be fairly different. 


#### Part F

Code up a **Q-learning** agent/algorithm to learn how to land the drone. You can do this however you like, as long as you use the MDP class structure defined above.  

Your code should include some kind of a wrapper to run many trials to train the agent and learn the Q values.  You also do not need to have a separate function for the actual "agent"; your code can just be a "for" loop within which you are refining your estimate of the Q values.

From each training trial, save the cumulative discounted reward (utility) over the course of that episode. That is, add up all of $\gamma^t R(s_t)$ where the drone is in state $s_t$ during time step $t$, for the entire sequence. We refer to this as "cumulative reward" because we usually refer to "utility" as the utility *under an optimal policy*.

Some guidelines:
* The drone should initialize in a random non-terminal state for each new training episode.
* The training episodes should be limited to 50 time steps, even if the drone has not yet landed. If the drone lands (in a terminal state), the training episode is over.
* You may use whatever learning rate $\alpha$ you decide is appropriate, and gives good results.
* There are many forms of Q-learning. You can use whatever you would like, subject to the reliability targets below.
* Your code should return:
  * The learned Q values associated with each state-action pair.
  * The cumulative reward for each training trial. 
  * Anything else that might be useful in the ensuing analysis.

In [6]:
from random import choice

#Returns an action 
def greedy(mdp, epsilon, s1, q, rewards):
    
    if np.random.random() < epsilon:
        return np.argmax(q[s1])
    else:
        #print(mdp.actions(s1))
        if s1 is tuple:
            return choice(mdp.actions(s1))
    #best_action = np.argmax(q[s1])
    #action_probs[best_action] += (1.0 - epsilon)
    #return action_probs

def Qlearning(mdp, num_episodes, learning_rate, epsilon):
    states = mdp.states
    states_actions = []
    #Make a {state: action } dictionary
    for s in states:
        actions = mdp.actions(s)
        if actions != []:
            for a in actions:
                states_actions.append((s, a))
        else:
            states_actions.append((s, ()))
            
    #Make another dictionary, add 0s
    Q = dict(zip(states_actions, [0]*len(states_actions)))
    #print(Q)
    rewards = {s: 0 for s in mdp.states}
    print("Rewards length:", len(rewards))
    print("States length:", len(mdp.states))
    
    rewards_arr = []
    
    for i in range(num_episodes): 
        #choose a random state for each episode        
    
        index = np.random.randint( 0,len(states) -1)
        #print(states[index])
        s1 = states[index]        
        cum_rewards = 0
        for j in range(50): 
            # If terminal state, calculate cum reward, exit
            if s1 in mdp.terminal_states:
                rewards[s1] += mdp.rewards[s1]
                cum_rewards += mdp.terminal_states[s1]
                break
            # Pick an action using e-greedy, calculate Q[s, a], update policy for that action
            else:
                #print(s1)
                a1 = greedy(mdp, epsilon, s1, Q, rewards)
                #print(len(rewards(s1)))
                #print(mdp.reward(s1), s1)         
                       
                #print("Setting reward ", s1, "=  ", mdp.reward(s1), "\n")
                rewards[s1] += mdp.reward(s1)
                cum_rewards += mdp.discount*j
                
                s2 = mdp.result(s1, a1)
                if mdp.actions(s2) == []:
                    a2 = ()
                else:
                    a2 = max(mdp.actions(s2), key = lambda a:Q[(s2, a)] - Q[(s1, a1)])
                    
                Q_curr = Q[(s1, a1)] + learning_rate * ( rewards[s1] + mdp.discount*(Q[(s2, a2)] - Q[(s1, a1)]))
                
                Q[(s1, a1)] = Q_curr
            #calculate cum_rewards
            rewards_arr.append(cum_rewards)
        return Q, rewards_arr

**I believe my MDPLanding class isn't working correctly, and there are too many moving parts here that I was not able to fix. That said I feel my Qlearning could have returned something useful; if it weren't for the MDPLanding problems.**

#### Part G

Initialize the $L=10$ environment (so that the landing pad is at $(5,5,0)$). Run some number of training trials to train the drone.

**How do I know if my drone is learned enough?**  If you take the mean cumulative reward across the last 5000 training trials, it should be around 0.80. This means at least about 10,000 (but probably more) training episodes will be necessary. It will take a few seconds on your computer, so start small to test your code.

**Then:** Compute block means of cumulative reward from all of your training trials. Use blocks of 500 training trials. This means you need to create some kind of array-like structure such that its first element is the mean of the first 500 trials' cumulative rewards; its second element is the mean of the 501-1000th trials' cumulative rewards; and so on. Make a plot of the block mean rewards as the training progresses. It should increase from about -0.5 initially to somewhere around +0.8.

**And:** Print to the screen the mean of the last 5000 trials' cumulative rewards, to verify that it is indeed about 0.80.

In [7]:
# Solution:
q_env = MDPLanding(L=10)
#q_env.actions((8, 4, 4))
Qlearning(q_env, num_episodes=1000, learning_rate=0.6, epsilon= 0.01)


Rewards length: 1331
States length: 1331


IndexError: tuple index out of range

#### Part H

**Question 1:** Why does the cumulative reward start off around -0.5 at the beginning of the training?

**Question 2:** Why will it be difficult for us to train the drone to reliably obtain rewards much greater than about 0.8?

**Your answer here:**
1. The drone in this case, performs 50 moves in the initial trials, the initial reward for each being -0.01. $\quad -0.01*50_{moves} = -0.5 $

2. Rewards of 0.8 become difficult to achieve with each successive training episode. 